In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

In [2]:
import tweepy as tw

# define keys
consumer_key= 'gEJhQtgiIvxzNB50u4JPic8f4'
consumer_secret= 'iEfTG65lFX8cAzKJ4QIhJklvuh3tfWdaRAAWO3b17082dZaSiu'
access_token= '1369691334051852293-IGWGrIUKFY6rTwmrA5WD3YLkJrlUk5'
access_token_secret= '7ftmxYiYnso7PNkPOWOWKCkNrguFFMhwwPTHhQ6bFVvgG'
# authenticate and create api object
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
import re

# just see how iterate through first 3 user's timeline pages with cursor
for page in tw.Cursor(api.user_timeline,
                      id="indykaila", exclude_replies=True,
                      include_rts=False, tweet_mode='extended').pages(3):
    for status in page:
        # preprocess documents (remove links and punctuations) to raw texts
        print(status.full_text)
        print(status.full_text.lower().split())
        link_removed = re.sub(r'\bhttps:\S+', '', status.full_text.lower())
        punc_link_removed = re.sub(r'[–,-.!":]\D', ' ', link_removed)
        print(punc_link_removed.split(), '\n---')

# Corpora

In [4]:
class MyTexts:
    """implement a generator object on a specific user timeline
    and preprocess (remove links and punctuations)
    to yield as row tokenized texts"""
    def __init__(self, pagination_num=3):
        self.pagination_num = pagination_num
        # cursor on user's timeline
        self.cursor = tw.Cursor(api.user_timeline, id="indykaila",
                              exclude_replies=True, include_rts=False,
                              tweet_mode='extended').pages(self.pagination_num)
    def __iter__(self):
        for page in self.cursor:
            for status in page:
                # cleaning: removing links and some punctuations
                link_removed = re.sub(r'\bhttps:\S+', '', status.full_text.lower())
                punc_link_removed = re.sub(r'[–,-.!":]\D', '', link_removed)
                yield punc_link_removed.split()

> Collect statistics about corpus, preprocess it and store training corpus

In [5]:
from gensim import corpora

texts = MyTexts(3)
# colloct statistics about all tokens
dictionary = corpora.Dictionary(texts)
# preprocess: remove stop words and only once words from dictionary
stop_words = set('for of a an and the to in'.split())
stop_word_ids = [dictionary.token2id[stopword]
                 for stopword in stop_words
                 if stopword in dictionary.token2id]
once_word_ids = [tokenid
                 for tokenid, docfreq in dictionary.dfs.items()
                 if docfreq == 1]
dictionary.filter_tokens(stop_word_ids + once_word_ids)
dictionary.compactify()
print(dictionary.dfs)
# store training corpus in bow representation for later use
# prefer work with generators (corpus streaming) instead of creating lists of documents
# train_texts = [text for text in MyTexts(3)]
train_texts = MyTexts(3)
'''note that every token that removed from dictionary is a "blah" for doc2bow method
and wouldn't count in corpus bow representation
but I use them in train_texts just for see the full texts somewhere'''
# train_corpus_bow = [dictionary.doc2bow(text) for text in train_texts]
train_corpus_bow = (dictionary.doc2bow(text) for text in train_texts)
corpora.MmCorpus.serialize('train_corpus_bow.mm', train_corpus_bow)

2021-04-01 13:32:06,546 : INFO : 'pattern' package not found; tag filters are not available for English
2021-04-01 13:32:06,728 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-04-01 13:32:07,147 : INFO : built Dictionary(330 unique tokens: ['a', 'be', 'due', 'for', 'injury#lfc']...) from 29 documents (total 547 corpus positions)
2021-04-01 13:32:07,153 : INFO : storing corpus in Matrix Market format to train_corpus_bow.mm
2021-04-01 13:32:07,159 : INFO : saving sparse matrix to train_corpus_bow.mm
2021-04-01 13:32:07,333 : INFO : PROGRESS: saving document #0


{3: 3, 4: 3, 0: 4, 2: 3, 1: 2, 6: 5, 5: 2, 7: 6, 8: 7, 10: 2, 15: 6, 12: 2, 13: 5, 16: 4, 19: 2, 20: 2, 18: 2, 9: 2, 11: 3, 17: 3, 21: 2, 14: 2, 23: 3, 25: 2, 24: 2, 22: 2, 29: 5, 26: 3, 28: 2, 27: 2, 31: 2, 30: 2, 32: 2, 35: 2, 33: 4, 36: 2, 37: 2, 34: 2, 38: 2, 39: 3, 40: 2, 41: 2, 42: 6, 44: 3, 45: 2, 43: 2, 48: 2, 47: 2, 46: 2, 50: 2, 51: 3, 49: 6, 52: 2, 54: 2, 53: 2, 57: 2, 55: 2, 59: 3, 56: 2, 58: 2, 60: 2, 62: 2, 63: 3, 61: 2, 64: 2, 65: 2, 66: 3}


2021-04-01 13:32:07,694 : INFO : saved 29x67 matrix, density=9.418% (183/1943)
2021-04-01 13:32:07,698 : INFO : saving MmCorpus index to train_corpus_bow.mm.index


In [ ]:
dictionary.dfs

In [ ]:
train_texts = MyTexts(3)
for text in train_texts:
    print(text)

In [ ]:
train_corpus_bow = corpora.MmCorpus('train_corpus_bow.mm')
for doc in train_corpus_bow:
    print(doc)

# Train the Models with Training Corpus

In [6]:
from gensim import models

# load training corpus in bow from disk
train_corpus_bow = corpora.MmCorpus('train_corpus_bow.mm')
# initialize a tfidf model with training corpus in bow: training
tfidf_model = models.TfidfModel(train_corpus_bow)
# tranform training corpus bow->tfidf
train_corpus_tfidf = tfidf_model[train_corpus_bow]
# train a LSI model with training corpus in tfidf
lsi_model = models.LsiModel(train_corpus_tfidf, id2word=dictionary, num_topics=2)
# tfidf->fold-in-lsi
train_corpus_lsi = lsi_model[train_corpus_tfidf]
train_corpus_lsi

2021-04-01 13:32:29,902 : INFO : loaded corpus index from train_corpus_bow.mm.index
2021-04-01 13:32:29,908 : INFO : initializing cython corpus reader from train_corpus_bow.mm
2021-04-01 13:32:29,914 : INFO : accepted corpus with 29 documents, 67 features, 183 non-zero entries
2021-04-01 13:32:29,918 : INFO : collecting document frequencies
2021-04-01 13:32:29,922 : INFO : PROGRESS: processing document #0
2021-04-01 13:32:29,924 : INFO : calculating IDF weights for 29 documents and 66 features (183 matrix non-zeros)
2021-04-01 13:32:29,926 : INFO : using serial LSI version on this node
2021-04-01 13:32:29,928 : INFO : updating model with new documents
2021-04-01 13:32:29,932 : INFO : preparing a new chunk of documents
2021-04-01 13:32:29,939 : INFO : using 100 extra samples and 2 power iterations
2021-04-01 13:32:29,941 : INFO : 1st phase: constructing (67, 102) action matrix
2021-04-01 13:32:29,948 : INFO : orthonormalizing (67, 102) action matrix
2021-04-01 13:32:29,969 : INFO : 2nd 

In [7]:
lsi_model.print_topics()

2021-04-01 13:32:34,501 : INFO : topic #0(1.690): 0.428*"is" + 0.330*"#nufc" + 0.200*"with" + 0.188*"at" + 0.185*"have" + 0.168*"out" + 0.163*"has" + 0.150*"no" + 0.149*"will" + 0.146*"want"
2021-04-01 13:32:34,502 : INFO : topic #1(1.537): 0.646*"#nufc" + 0.425*"is" + -0.225*"news" + -0.182*"all" + -0.135*"at" + -0.129*"with" + -0.121*"chance" + -0.112*"manchester" + 0.110*"want" + -0.105*"no"


[(0,
  '0.428*"is" + 0.330*"#nufc" + 0.200*"with" + 0.188*"at" + 0.185*"have" + 0.168*"out" + 0.163*"has" + 0.150*"no" + 0.149*"will" + 0.146*"want"'),
 (1,
  '0.646*"#nufc" + 0.425*"is" + -0.225*"news" + -0.182*"all" + -0.135*"at" + -0.129*"with" + -0.121*"chance" + -0.112*"manchester" + 0.110*"want" + -0.105*"no"')]

In [8]:
train_texts = MyTexts(3)
for doc_lsi, as_text in zip(train_corpus_lsi, train_texts):
    print(doc_lsi, as_text)

[(0, 0.2634753900441857), (1, -0.06353021807784559)] ['sergio', 'ramos', 'set', 'to', 'be', 'out', 'for', 'a', 'month', 'due', 'to', 'injury#lfc']
[(0, 0.3787829963532377), (1, -0.14531977919418487)] ['bayern', 'munich', 'have', 'confirmed', 'robert', 'lewandowski', 'will', 'be', 'out', 'for', '4', 'weeks', 'with', 'a', 'knee', 'injury.']
[(0, 0.34944563689622904), (1, -0.250849484693159)] ['sergio', 'aguero', 'has', 'been', 'advised', 'by', 'his', 'public', 'relations', 'team', 'not', 'to', 'move', 'to', '@chelseafc', '@fcbarcelona', 'are', 'leading', 'the', 'racetalks', 'to', 'start', 'next', 'week', 'with', 'the', 'spanish', 'club.']
[(0, 0.24920297656699164), (1, -0.01482318304162268)] ['if', 'you', 'want', 'to', 'make', 'some', 'extra', 'money', 'tomorrow', 'give', '@betciub', 'a', 'followthey', 'have', 'some', 'huge', 'winners', '💷', '#ad']
[(0, 0.25317608579162876), (1, -0.1780255774122605)] ['manchester', 'city', 'confirm', 'sergio', 'aguero', 'will', 'leave', 'club', 'this', '